In [1]:
"""
NOTE:
Before running this, create a virtual environment running
Python 3.12
""" 
# pytorch install, cpu-only
%pip install torch 
%pip install transformers
%pip install sentence-transformers
%pip install pandas
%pip install chonkie
%pip install pypdf[image]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [1]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import StaticEmbedding

import os
import sys
import pandas as pd
import numpy as np
# Add the current directory to the path
sys.path.append(os.getcwd())
from preprocess_fast import prepare_PDF

c:\Users\Steven\Documents\Python\super-search\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define paths
main = r"C:\Users\Steven\Documents\Python\super-search"
data = f"{main}/data/tests"
test_file = f"{data}/32286.pdf"

In [3]:
df = pd.DataFrame.from_dict(prepare_PDF(test_file))
df

,raw_chunk,processed_chunk,file_path
0,NBER WORKING PAPER SERIES\nPROMOTING PUBLIC HE...,PROMOTING PUBLIC HEALTH WITH BLUNT INSTRUMENTS...,C:\Users\Steven\Documents\Python\super-search/...
1,"32286\nMarch 2024\nJEL No. H70,I1,I11,J20\nABS...","32286 March 2024 JEL No. H70,I1,I11,J20 study ...",C:\Users\Steven\Documents\Python\super-search/...
2,"drawn to healthcare, relaxing shortages. On th...","drawn to healthcare, relaxing shortages. On ot...",C:\Users\Steven\Documents\Python\super-search/...
3,employment \ntransitions into and out of the i...,employment transitions into out industry. find...,C:\Users\Steven\Documents\Python\super-search/...
4,workers in non-healthcare occupations. Finding...,workers in non-healthcare occupations. Finding...,C:\Users\Steven\Documents\Python\super-search/...
...,...,...,...
254,high school 0.086 0.025 0.025 0.024\nHigh scho...,high school 0.086 0.025 0.025 0.024 High schoo...,C:\Users\Steven\Documents\Python\super-search/...
255,0.99 0.99\nEmployed 0.72 0.97 0.97 0.97\nFull-...,0.99 0.99 Employed 0.72 0.97 0.97 0.97 Full-ti...,C:\Users\Steven\Documents\Python\super-search/...
256,"variables IND and OCC, which provide informati...","variables IND OCC, which provide information m...",C:\Users\Steven\Documents\Python\super-search/...
257,pre-mandate states\nExitt or Entert 0.0568 0.0...,pre-mandate states Exitt or Entert 0.0568 0.05...,C:\Users\Steven\Documents\Python\super-search/...


In [4]:
prepare_PDF(test_file)['processed_chunk'][0]

'PROMOTING PUBLIC HEALTH WITH BLUNT INSTRUMENTS EVIDENCE FROM VACCINE MANDATES Rahi Abouk John S. Earle Johanna Catherine Maclean Sungbin Park Working Paper 32286 http //www.nber.org/papers/w32286 1050 Massachusetts Avenue Cambridge, MA 02138 March 2024 Research reported in publication supported National Institute on Mental Health National Institutes Health under Award Number 1R01MH132552 (PI Johanna Catherine Maclean). John Earle also acknowledges support Russell Sage Foundation. views expressed herein authors not necessarily reflect views National Institutes Health or. NBER working papers circulated discussion comment purposes. not peer-reviewed or subject to review NBER Board Directors accompanies official NBER publications. 2024 Rahi Abouk, John S. Earle, Johanna Catherine Maclean, Sungbin Park. All rights reserved. Short sections text, not to exceed two paragraphs, may quoted without explicit permission provided full credit, including notice, given to source. Promoting Public Heal

## To-Do:

### Database management
- We need a system to handle vector database as well as allowing for fast retrieval of the files corresponding to each vector
    - Each file needs a file_id
        - Links to the filepath
    - Each chunk needs a chunk_id
        - Links to the chunk text
            - ~~Importantly, link to the original text, not the processed text used for embedding.~~
                - ~~**I think the solution is to do the sentence chunking on the original text, before processing**~~
    - file_id + chunk_id should uniquely identify a vector in the database
- After taking a user query and encoding it, perform similarity search in the database
    to identify a row, then link to the row's filepath and text
- Then print a hyperlink to the file, and print the original text
- **Problem**: how to return images, given that we first caption them with an LLM and then encode the caption?

### Misc.
- Parallelized PDF processing
- Timing everything to understand where the bottlenecks are
- Save performance statistics to report in the app
    - How many PDF pages have been read? 

### Incorporating Images
- Use PyPDF to extract images from each page of the PDF.
```
for page in reader.pages:
    try:
        for image in page.images:
            extract_image(image)
    except:
        pass
```
- To render the image visually: image.image. Can also show as bytes
- Goal is to pass each image to a multi-modal LLM for summary, which is then fed into the cleaned text.
    - Possibly LLaVA for describing the images.
    - Since captioning the images would take a super long time, this should be an optional step.
        - Ideally would be done after first parsing the text, but then you might have to regenerate the whole vector base

### GUI
- The following should be customizable inputs:
    - Chunk token size ("larger is faster but less accurate") (default: 256)
    - Chunk overlap ("larger gives more context per chunk") (default: chunk_size / 4)
    - Choice of sentence transformer: provide a few options based on speed/accuracy tradeoff.
        - Fastest: static-retrieval-mrl-en-v1
        - Medium: bge-m3
        - Slowest: gte-large-en-v1.5
        - (these are subject to change)
    - Index database save location
    - Similarity matrix (default: cosine)

In [5]:
# Current encoding model implementation: static-retrieval-mrl-en-v1
# https://huggingface.co/sentence-transformers/static-retrieval-mrl-en-v1
# Model defaults to 1024 dense dimensions, but can be truncated to save space/time

truncated_dimensions = 1024

model = SentenceTransformer(
    "sentence-transformers/static-retrieval-mrl-en-v1"
    , device="cpu"
    , truncate_dim=truncated_dimensions
    )


In [13]:
vecs = model.encode(df['processed_chunk'])
# This returns a np array of shape (n, d), where n is 
#     number of chunks and d is embedding dimensions.

df2['vector'] = [i for i in np.unstack(vecs)]
# Add the embeddings to our dataframe in a single variable,
#     so each cell contains the d-dimensional np vector.

In [7]:
## TESTING
# Importing a lot of PDFs to see how long this takes
papers_repo = r"C:\Users\Steven\Documents\Python\Data\NBER papers"

files = os.listdir(papers_repo)
files.sort(reverse=True)

full_dict = {
    'raw_chunk': []
    , 'processed_chunk': []
    , 'file_path': []
}

counter=1

for paper in files[0:99]:
    f = f"{papers_repo}/{paper}"
    iter_dict = prepare_PDF(f)
    full_dict['raw_chunk'].extend(iter_dict['raw_chunk'])
    full_dict['processed_chunk'].extend(iter_dict['processed_chunk'])
    full_dict['file_path'].extend(iter_dict['file_path'])
    counter+=1
    if counter%5==0:
        print(f"Finished filed {counter}.")

df = pd.DataFrame.from_dict(full_dict)
df

# Currently takes around 1 second per file (with tokenization chunking)
# Takes ~ 0.7 seconds with approximate chunking
    

Finished filed 5.
Finished filed 10.
Finished filed 15.
Finished filed 20.
Finished filed 25.
Finished filed 30.
Finished filed 35.
Finished filed 40.
Finished filed 45.
Finished filed 50.
Finished filed 55.
Finished filed 60.
Finished filed 65.
Finished filed 70.
Finished filed 75.
Finished filed 80.
Finished filed 85.
Finished filed 90.
Finished filed 95.
Finished filed 100.


,raw_chunk,processed_chunk,file_path
0,NBER WORKING PAPER SERIES\nRETIREMENT AND THE ...,RETIREMENT AND THE EVOLUTION OF PENSION STRUCT...,C:\Users\Steven\Documents\Python\Data\NBER pap...
1,in American retirement ages. In this paper we ...,in American retirement ages. In paper find abs...,C:\Users\Steven\Documents\Python\Data\NBER pap...
2,VA 22904-4182\nand NBER\nlfriedberg@virginia.e...,VA 22904-4182 NBER lfriedberg virginia.edu Ant...,C:\Users\Steven\Documents\Python\Data\NBER pap...
3,from 87% in 1983 to 44% in 1998.1 \nPension w...,from 87% in 1983 to 44% in 1998.1 Pension weal...,C:\Users\Steven\Documents\Python\Data\NBER pap...
4,to stay early on in order to gain access to \n...,to stay early on in order to gain access to la...,C:\Users\Steven\Documents\Python\Data\NBER pap...
...,...,...,...
20506,35\nTable 4: The Effect of Birth Weight on Po...,35 Table 4 Effect Birth Weight on Post-neonata...,C:\Users\Steven\Documents\Python\Data\NBER pap...
20507,(.059) (.111) (.071) (.094) (.108)...,(.059) (.111) (.071) (.094) (.108) Female -.31...,C:\Users\Steven\Documents\Python\Data\NBER pap...
20508,(.141) N 2364 698 1666 940 726 ...,(.141) N 2364 698 1666 940 726 ***p<.001 **p<....,C:\Users\Steven\Documents\Python\Data\NBER pap...
20509,\nparentheses (See Pps. 18-19 for formula used...,parentheses (See Pps. 18-19 formula used to de...,C:\Users\Steven\Documents\Python\Data\NBER pap...


In [ ]:
vecs = model.encode(df['processed_chunk'])
# This returns a np array of shape (n, d), where n is 
#     number of chunks and d is embedding dimensions.

df['vector'] = [i for i in np.unstack(vecs)]
# Add the embeddings to our dataframe in a single variable,
#     so each cell contains the d-dimensional np vector.

df
# takes 3-4 seconds

,raw_chunk,processed_chunk,file_path,vector
0,NBER WORKING PAPER SERIES\nRETIREMENT AND THE ...,RETIREMENT AND THE EVOLUTION OF PENSION STRUCT...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[0.6509546, 0.6684445, 0.7835084, -2.9097886, ..."
1,in American retirement ages. In this paper we ...,in American retirement ages. In paper find abs...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[0.24004129, 1.8614491, -0.5491161, -3.0589666..."
2,VA 22904-4182\nand NBER\nlfriedberg@virginia.e...,VA 22904-4182 NBER lfriedberg virginia.edu Ant...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[1.10191, 1.8557167, -0.7971541, -3.4970014, 0..."
3,from 87% in 1983 to 44% in 1998.1 \nPension w...,from 87% in 1983 to 44% in 1998.1 Pension weal...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[1.676269, 0.9347222, -1.3195981, -2.66524, 1...."
4,to stay early on in order to gain access to \n...,to stay early on in order to gain access to la...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[1.1903515, 0.18316549, -1.7624729, -2.7970293..."
...,...,...,...,...
20506,35\nTable 4: The Effect of Birth Weight on Po...,35 Table 4 Effect Birth Weight on Post-neonata...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[-0.15989622, -1.6293253, 2.398079, -0.2228098..."
20507,(.059) (.111) (.071) (.094) (.108)...,(.059) (.111) (.071) (.094) (.108) Female -.31...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[0.58949625, -1.708889, 2.3602786, -0.1460202,..."
20508,(.141) N 2364 698 1666 940 726 ...,(.141) N 2364 698 1666 940 726 ***p<.001 **p<....,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[0.068958014, -2.090059, 2.953155, 0.18803951,..."
20509,\nparentheses (See Pps. 18-19 for formula used...,parentheses (See Pps. 18-19 formula used to de...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[-0.6008663, -1.7976931, 2.1618726, 0.57339424..."
